In [ ]:
# Calvano et al. (2020) Q-Learning Replication (Colab)

このノートブックは Calvano et al. (2020) の Q学習による価格付け実験を Google Colab 上で再現します。

**論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). Artificial Intelligence, Algorithmic Pricing, and Collusion. *American Economic Review*, 110(10), 3267-3297.

---

## 📋 実行手順

1. **セットアップ**: セクション1-2を順番に実行
2. **ベンチマーク計算**: セクション3で理論値を確認
3. **クイックテスト**: セクション4で基本動作確認（100エピソード、約5分）
4. **結果確認**: セクション5で学習結果を可視化
5. **フル実験**: セクション6で統計的に有意な結果を取得
   - **推奨**: 中規模実験（10,000エピソード × 3シード、3-4時間）
   - **完全版**: フル実験（50,000エピソード × 10シード、10-15日）

**💡 推奨フロー**: セクション1→2→3→4→5→6（中規模実験）


In [ ]:
## 1. セットアップ（リポジトリのクローン＆依存パッケージのインストール）


In [ ]:
# リポジトリをクローン（並列実行対応ブランチ）
!git clone -b parallel-colab https://github.com/Yusei406/calvano-thesis.git
%cd calvano-thesis

# 現在のディレクトリを確認
!pwd
!ls -la

# 依存パッケージをインストール
!pip install -r requirements.txt

# パッケージが正しくインストールされたか確認
import sys
import os
print(f"Python path: {sys.path}")

# 現在の作業ディレクトリを取得
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# myprojectディレクトリの存在確認
if os.path.exists('myproject'):
    print("✅ myprojectディレクトリが見つかりました")
    print("📁 myproject内容:")
    !ls -la myproject/
else:
    print("❌ myprojectディレクトリが見つかりません")
    print("📁 現在のディレクトリ内容:")
    !ls -la

print("✅ セットアップ完了")


In [ ]:
## 2. モジュールのインポート


In [ ]:
import sys
import os

# 現在のディレクトリをPythonパスに追加
sys.path.append('.')
sys.path.append(os.getcwd())

print(f"📍 現在のディレクトリ: {os.getcwd()}")
print(f"📁 myprojectの存在確認: {os.path.exists('myproject')}")

# 必要なモジュールをインポート
try:
    from myproject.env import DemandEnvironment
    from myproject.agent import QLearningAgent
    from myproject.train import train_agents
    print("✅ myprojectモジュールのインポート成功")
except ImportError as e:
    print(f"❌ myprojectモジュールのインポートエラー: {e}")
    print("📋 利用可能なファイル:")
    !find . -name "*.py" | head -10
    raise

# 標準ライブラリとサードパーティライブラリ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print("✅ 全モジュールのインポート完了")


In [ ]:
## 3. 環境セットアップとベンチマーク計算（Nash/協調均衡）


In [ ]:
# Calvanoパラメータで環境を作成
env = DemandEnvironment(
    demand_intercept=0.0,   # a_0 (outside option)
    product_quality=2.0,    # a_i (product quality)
    demand_slope=0.25,      # μ (price sensitivity)
    marginal_cost=1.0       # c (marginal cost)
)

# Nash均衡・協調均衡を計算
nash_eq = env.get_nash_equilibrium()
coop_eq = env.get_collusive_outcome()

print('📊 ベンチマーク均衡の計算結果:')
print()
print('Nash Equilibrium:')
print(f'  Price: {nash_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {nash_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {nash_eq["joint_profit"]:.3f}')
print()
print('Cooperative Equilibrium:')
print(f'  Price: {coop_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {coop_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {coop_eq["joint_profit"]:.3f}')


In [ ]:
## 4. Q学習エージェント学習（クイックテスト）

**クイックテスト専用**: 基本動作確認とアルゴリズムのテスト用
- **パラメータ**: 100エピソード × 1,000イテレーション
- **実行時間**: 約5分
- **目的**: 協調的行動の基本確認

**注意**: フル実験は後のセクション6で実行します


In [ ]:
# クイックテスト（基本動作確認用）
print('🚀 クイックテスト実験を開始...')
print('📊 パラメータ: 100エピソード × 1,000イテレーション')
print('⏰ 予想実行時間: 約5分')
print()

agents, env, history = train_agents(
    n_episodes=100,
    iterations_per_episode=1000,
    learning_rate=0.15,
    discount_factor=0.95,
    epsilon_decay_beta=4e-6,
    memory_length=1,
    verbose=True
)

print('\n✅ クイックテスト完了!')
print(f'Final individual profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Final joint profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Nash ratio (individual): {history["training_summary"]["nash_ratio_individual"]:.3f}')
print()
print('💡 このテストで協調的行動（Nash比>100%）が確認できれば、基本実装は正常です。')
print('📊 論文レベルの統計的に有意な結果を得るには、セクション6のフル実験を実行してください。')


In [ ]:
## 5. 学習履歴の可視化


In [ ]:
# 学習結果のプロット
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 個別利益の推移
axes[0, 0].plot(history['episodes'], history['individual_profits'], 'b-', linewidth=2)
axes[0, 0].axhline(y=nash_eq['individual_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 0].axhline(y=coop_eq['individual_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 0].set_title('Individual Profits', fontsize=14)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Profit')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 合計利益の推移
axes[0, 1].plot(history['episodes'], history['joint_profits'], 'b-', linewidth=2)
axes[0, 1].axhline(y=nash_eq['joint_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 1].axhline(y=coop_eq['joint_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 1].set_title('Joint Profits', fontsize=14)
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Profit')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Nash比
nash_ratios = [p / nash_eq['individual_profit'] for p in history['individual_profits']]
axes[1, 0].plot(history['episodes'], nash_ratios, 'orange', linewidth=2)
axes[1, 0].axhline(y=1.0, color='r', linestyle='--', label='Nash', linewidth=2)
axes[1, 0].set_title('Individual Profit / Nash Profit', fontsize=14)
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Epsilon減衰
axes[1, 1].plot(history['episodes'], history['epsilon_values'], 'purple', linewidth=2)
axes[1, 1].set_title('Epsilon Decay', fontsize=14)
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Epsilon')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('📈 最終結果:')
print(f'Individual Profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Joint Profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Nash Ratio: {history["training_summary"]["nash_ratio_individual"]:.3f}')


In [ ]:
## 6. フル実験（Table A.2完全再現）

**論文レベルの統計的に有意な結果**を得るための本格実験：

**実験レベルの選択**:
- **中規模実験**: 10,000エピソード × 3シード（約3-4時間）- 推奨
- **フル実験**: 50,000エピソード × 10シード（約10-15日）- 論文完全再現

**注意**: フル実験は非常に長時間かかります。まず中規模実験で結果を確認することを推奨します。


In [ ]:
# 並列実験スクリプトの存在確認と実行
import os
from datetime import datetime

# 並列実験スクリプトの存在確認
script_path = 'myproject/scripts/table_a2_parallel.py'
if os.path.exists(script_path):
    print(f"✅ 並列実験スクリプトが見つかりました: {script_path}")
    print()
    
    # 実験レベルの選択
    EXPERIMENT_TYPE = "medium"  # "small", "medium", "full"
    
    if EXPERIMENT_TYPE == "small":
        # 小規模実験（約30分）
        print("🚀 小規模実験を開始...")
        print("📊 パラメータ: 1,000エピソード × 3シード")
        print("⏰ 予想実行時間: 30分")
        episodes, seeds, sessions = 1000, 3, 4
        
    elif EXPERIMENT_TYPE == "medium":
        # 中規模実験（約3-4時間）
        print("🚀 中規模実験を開始...")
        print("📊 パラメータ: 10,000エピソード × 3シード")
        print("⏰ 予想実行時間: 3-4時間")
        episodes, seeds, sessions = 10000, 3, 4
        
    else:  # full
        # フル実験（非推奨 - Colabでは実行困難）
        print("⚠️  フル実験はColabでは推奨されません")
        print("📊 パラメータ: 50,000エピソード × 10シード")
        print("⏰ 予想実行時間: 20+ 時間（セッション切断リスク高）")
        episodes, seeds, sessions = 50000, 10, 4
    
    print(f"⏰ 開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # 並列実験を実行（コメントアウトを外して使用）
    # !python -m myproject.scripts.table_a2_parallel --episodes {episodes} --n-seeds {seeds} --n-sessions {sessions} --max-workers 4 --verbose
    
    print("💡 上記のコメントアウトを外して実行してください。")
    print("⚠️  実験には時間がかかるため、Colab Pro/Pro+の使用を推奨します。")
        
else:
    print(f"❌ 並列実験スクリプトが見つかりません: {script_path}")
    print("📋 利用可能なスクリプト:")
    !find myproject -name "*.py" -type f


In [ ]:
---

## 📚 参考情報

### 期待される結果（論文Table A.2）
- **Individual profit**: 0.18 ± 0.03
- **Joint profit**: 0.26 ± 0.04
- **Nash ratio**: > 100% (協調的行動を示唆)

### パラメータ設定
- **環境**: a₀=0.0, aᵢ=2.0, μ=0.25, c=1.0
- **Q学習**: α=0.15, γ=0.95, β=4×10⁻⁶, memory=1

### 実行時間の目安
- **クイックテスト**: 100エピソード（約5分）
- **中規模実験**: 10,000エピソード × 3シード（約3-4時間）
- **フル実験**: 50,000エピソード × 10シード（約10-15日）

**🎓 論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). *Artificial Intelligence, Algorithmic Pricing, and Collusion*. American Economic Review, 110(10), 3267-3297.


In [ ]:
## 🔧 トラブルシューティング

### よくあるエラーと解決方法

**1. `ModuleNotFoundError: No module named 'myproject'`**
- セクション1のセットアップセルを正しく実行したか確認
- `%cd calvano-thesis` が正しく動作しているか確認

**2. `requirements.txt` インストールエラー**
- Colab環境で必要なパッケージが既にインストール済みの場合があります
- エラーが出た場合は個別にインストール: `!pip install numpy scipy matplotlib pandas tqdm`

**3. 並列実験エラー**
- 並列実験はオプションです。基本実験（セクション4-5）は単体で動作します

**4. セッションタイムアウト**
- 長時間実験はColab Pro/Pro+の使用を推奨
- 実験中はタブを閉じないでください
